## Angle Calculations Testing 

In [1]:
%matplotlib qt
# inline

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
import random
from tqdm import tqdm
import pandas as pd

from AngleHelpers import calculate_thetas_from_unity_coords, calculate_rotation_matrix, unity_pts_to_right_handed_cartesian_coords, \
                    right_handed_cartesian_coords_to_unity_pts, angle_between, angle_in_180s, cart2sph, sph2cart, deg2rad, rad2deg, \
                    create_relative_direction, create_relative_directions, create_relative_direction_consider_roll, \
                    create_relative_directions_consider_roll,calculate_norm_dir_from_unity_angles
# Methods: - "anglediff" - "anglediff_sphere_coords" - "unit_sphere_rotation"


# Information on Angles in Unity and its conversions
Coordinate Systems (see also cell below)
* Unity coordinate system: Left-handed, i.e. rotations come about by pointing the left hand's thumb in the direction of the axis and the other fingers wrapped around the axis point in the direction where the angle increases.
* Typical math coordinates: Right-handed, same procedure as above with right-hand; the axes have a different setup in comparison to Unity.
* From one system's axis to the other's: Unity -> The one used here: x -> x, y -> z, z -> y

From angles and directions:
* A direction vector gives the "look-direction" of an object. It includes info about yaw and pitch, but roll is omitted, as that would be just a rotation of the direction vector around itself.
* Rotation angles give the rotation of the object, by rotating it around the main axis one after another of its components (x,y,z). There's a distinction between extrinsic and intrinsic rotation. Extrinsic rotations assume that the "world coordinate system around the object" stays the same while the object is rotated inside. Intrinsic rotations assume that the frame of reference (the coordinate system) of the object moves with it as it is rotated in space (i.e. relative to the object the coordinate system stays the same; changing the rotation values of an object in the Unity editor affects the object intrinsically). 
* Unity angles: Unity internally represents rotations as Quaternions, but converting the Quaternions to angles using Quaternion.eulerAngles, produces x,y,z angles, that when applied to the object in the order z,x,y extrinsically (order and reference frame important) produce the desired rotation. Source: https://docs.unity3d.com/ScriptReference/Quaternion-eulerAngles.html 
* (Untested so far: According to a post on the Unity forum, y,x,z intrinsically produces the same result. https://forum.unity.com/threads/which-euler-angles-convention-used-in-unity.41114/#post-6828797 )

Converting angles to direction:
* To calculate a direction vector from rotation angles in Unity, `Quaternion.Euler(theVectorAngles) * Vector3.forward;` can be used. Source: https://forum.unity.com/threads/convert-a-vector3-containing-eulerangles-into-a-normalized-direction-vector3.124171/#post-846168 
* The calculations can be replicated in Python, compare the `calculate_norm_dir_from_unity_angles` function in the AngleHelpers script. The initial code was changed such that a minus was added to the angle in the calculations of the elevation, to ensure correct results that are the same as with the Unity method. Original source: https://forum.unity.com/threads/convert-a-vector3-containing-eulerangles-into-a-normalized-direction-vector3.124171/#post-846164 

Rotating directions to other directions:
* When using only directions, the roll of the direction vectors is not known. In so far, the result will be mathematically correct, but will not include any consideration of the roll of the direction vector.
* Optionally, it is possible to use the rotations that lead to the construction of the direction vector and "inverse" apply them to another direction vector. That way, the information of roll of the first direction vector is kept and accounted for in the shift of the second direction vector.
* All approaches are described in more detail below.


In [2]:
# Coordinate Systems

'''

# Typically in math, used here as well (used here as right-handed)

    /\ z
    |
    | /\ y
    | /
    |/------->   x
    /
   /
   

# Unity (left-handed) 

    /\ y
    |
    |  /
    | /
    |------->   z
    /
   /
  /
 \/ x
 

Unity -> The one used here 
x -> x 
y -> z 
z -> y 

'''

'\n\n# Typically in math, used here as well (used here as right-handed)\n\n    /\\ z\n    |\n    | /\\ y\n    | /\n    |/------->   x\n    /\n   /\n   \n\n# Unity (left-handed) \n\n    /\\ y\n    |\n    |  /\n    | /\n    |------->   z\n    /\n   /\n  /\n \\/ x\n \n\nUnity -> The one used here \nx -> x \ny -> z \nz -> y \n\n'

# Unifying method for multiple directions that considers reference roll

In [3]:
print("Using the unifying method for multiple relative directions - Considers reference roll")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Using the unifying method for multiple relative directions, single result - Considers reference roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')


# Create dummy data 
df = pd.DataFrame(columns=["inp_x","inp_y","inp_z","ref_angle_x","ref_angle_y","ref_angle_z","final_x","final_y","final_z"])
df["inp_x"] = [-1,0.5,0.5]
df["inp_y"] = [0,-0.2,-0.2]
df["inp_z"] = [1,0.3,0.5]
df["ref_angle_x"] = [45,0,-30]
df["ref_angle_y"] = [-45,70,-20]
df["ref_angle_z"] = [45,90,20]

# Calculate relative directions
final_x, final_y, final_z = create_relative_directions_consider_roll(df["inp_x"],df["inp_y"],df["inp_z"],df["ref_angle_x"],df["ref_angle_y"],df["ref_angle_z"])
df["final_x"] = final_x
df["final_y"] = final_y
df["final_z"] = final_z


print("Resulting directions")
print(df)

# Plot single result
plot_idx = 0
ref_dir_unity = calculate_norm_dir_from_unity_angles(df["ref_angle_x"][plot_idx],df["ref_angle_y"][plot_idx],df["ref_angle_z"][plot_idx])
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,df["inp_x"][plot_idx],df["inp_z"][plot_idx],df["inp_y"][plot_idx],color="green") # input, change Unity coords to cartesian right hand
ax1.quiver(0,0,0,ref_dir_unity[0],ref_dir_unity[2],ref_dir_unity[1],color="blue") # reference, change Unity coords to cartesian right hand
ax1.quiver(0,0,0,df["final_x"][plot_idx],df["final_z"][plot_idx],df["final_y"][plot_idx],color="red") # final, change Unity coords to cartesian right hand


plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()



   

Using the unifying method for multiple relative directions - Considers reference roll


100%|██████████| 3/3 [00:00<00:00, 253.83it/s]

Creating relative directions considering roll.
Resulting directions
   inp_x  inp_y  inp_z  ref_angle_x  ref_angle_y  ref_angle_z   final_x  \
0   -1.0    0.0    1.0           45          -45           45  0.500000   
1    0.5   -0.2    0.3            0           70           90 -0.324443   
2    0.5   -0.2    0.5          -30          -20           20  0.669343   

    final_y   final_z  
0  0.500000  0.707107  
1  0.179900  0.928640  
2 -0.710832  0.216099  


# Unifying method for single direction that considers reference roll

In [4]:
print("Using the unifying method for single relative direction - Consider Roll")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Using the unifying method for single relative direction - Consider Roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# from Unity angles to direction 
unity_angle_x = 45  
unity_angle_y = -45 
unity_angle_z = 45
ref_dir_norm = calculate_norm_dir_from_unity_angles(unity_angle_x,unity_angle_y,unity_angle_z)

# Input and reference 
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array(ref_dir_norm)

# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]

# Get output 
ref_angles = (unity_angle_x,unity_angle_y,unity_angle_z)
final_dir_unity = create_relative_direction_consider_roll(inp_dir_unity,ref_angles,log_verbose=True)
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]


# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final rotated

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Using the unifying method for single relative direction - Consider Roll
Reference horizontal: -44.999999999999986
Reference vertical: -44.999999999999986
Input horizontal: -45.00000000000001
Input vertical: 1.2074182697257333e-06
Rotated reference: [0. 1. 0.]
Rotated reference horizontal: -0.0
Rotated reference vertical: 0.0
Rotated input: [0.70710678 1.         0.70710678]
Rotated input horizontal: 35.26438968275465
Rotated input vertical: 30.000000000000004
Final direction vector: [0.49999999999999983, 0.5000000000000001, 0.7071067811865476]


# Approach that considers roll of reference
Background on actual data: Frame of reference matter. The eye direction (and nose direction) should be in the frame of reference of the car. I.e. rotate the car direction (which is in world frame of reference) such that the axis of the car coordinate system and the world coordinate system align. Then apply the same transformation to the eye and nose direction (i.e. the transformation maps from world -> car frame of reference). This idea is the same with the approaches below. However, roll of the car needs to be taken into account as well, i.e. the rotation around the axis it is aligned from front to back on. 
This approach takes rotations around all axis into account to provide a full transformation of frame of references.

On the rotation: 
According to Unity Docs, Euler Angles represent a rotation "that rotates z degrees around the z axis, x degrees around the x axis, and y degrees around the y axis; applied in that order" extrinsically. https://docs.unity3d.com/ScriptReference/Quaternion.Euler.html
Thus construct rotation matrices that rotate the same angles back in inversed order to end up at the unity forward direction.

In [2]:
# %%timeit

# Version 3 -- CONSIDERS ROLL OF REFERENCE
# Take Unity angles (order of application is z, x, y extrinsical) that lead to the creation of the direction.
# Use these angles to create rotation matrices that inverse those rotations. 
print("Version 3 - Inverse the angles that lead to the direction vector")

# Init Plot
fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 3 - Inverse the angles that lead to the direction vector")
ax1 = fig.add_subplot(1,1,1, projection='3d')


# From Unity angles to direction 
unity_angle_x = 45  
unity_angle_y = -45
unity_angle_z = 45
dirvec_norm = calculate_norm_dir_from_unity_angles(unity_angle_x,unity_angle_y,unity_angle_z)


# Input and reference 
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array(dirvec_norm) # only used for plotting purposes 


# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]


# Calculate horizontal angle offset as angle between direction vector's projection on x,y plane (cartesian) and forward vector
# Alternatively, use calculate_thetas_from_unity_coords (but can only use horizontal value here, because projection would skew vertical value)
ref_horizontal = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0])

# Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if ref_dir_cartesian[2] < 0:
    ref_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if ref_dir_cartesian[0] < 0:
    ref_horizontal *= -1
    
# Calculate horizontal angle offset for input 
inp_horizontal = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0])

# Calculate vertical angle offset for input
inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if inp_dir_cartesian[0] < 0:
    inp_horizontal *= -1


print("Reference horizontal: " + str(ref_horizontal))
print("Reference vertical: " + str(ref_vertical))
print("Input horizontal: " + str(inp_horizontal))
print("Input vertical: " + str(inp_vertical))

# Calculate rot matrices, Axis are for right-handed coordinate system
# Unity original order of applied angles: z, x, y 
# Inverse Unity order: y, x, z 
# Inverse Unity order in cartesian: z, x, y 
# Use negative value of angles 

horizontal_angle = unity_angle_y
vertical_angle = unity_angle_x 
around_itself_angle = unity_angle_z 
horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle = horizontal_angle, angle_in_degree=True)
vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle = vertical_angle, angle_in_degree=True)
around_itself_rot_mat = calculate_rotation_matrix(rotate_around_axis="y", angle = around_itself_angle, angle_in_degree=True)

# Apply calculated rotations to reference direction, in order to rotate it to forward direction (Unity 0,0,1)
# Keep order of rotations!
ref_first_rot = ref_dir_cartesian.copy()
ref_first_rot = np.dot(horizontal_rot_mat,ref_first_rot)
ref_second_rot = np.dot(vertical_rot_mat, ref_first_rot)
rotated_ref = np.dot(around_itself_rot_mat, ref_second_rot)


# Calculate horizontal angle offset of rotated reference
rotated_ref_horizontal = angle_between([rotated_ref[0],rotated_ref[1],0],[0,1,0])

# Calculate vertical angle offset of rotated reference
rotated_ref_vertical = angle_between(rotated_ref,[rotated_ref[0],rotated_ref[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if rotated_ref[2] < 0:
    rotated_ref_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if ref_dir_cartesian[0] < 0:
    rotated_ref_horizontal *= -1

print("Rotated reference: " + str(rotated_ref))
print("Rotated reference horizontal: " + str(rotated_ref_horizontal))
print("Rotated reference vertical: " + str(rotated_ref_vertical))



# Apply calculated rotations to input direction
# Same order, as for reference!
inp_first_rot = inp_dir_cartesian.copy()
inp_first_rot = np.dot(horizontal_rot_mat,inp_first_rot)
inp_second_rot = np.dot(vertical_rot_mat, inp_first_rot)
rotated_inp = np.dot(around_itself_rot_mat, inp_second_rot)


# Calculate horizontal angle offset of rotated input
rotated_inp_horizontal = angle_between([rotated_inp[0],rotated_inp[1],0],[0,1,0])

# Calculate vertical angle offset of rotated input
rotated_inp_vertical = angle_between(rotated_inp,[rotated_inp[0],rotated_inp[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if rotated_inp[2] < 0:
    rotated_inp_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if rotated_inp[0] < 0:
    rotated_inp_horizontal *= -1

print("Rotated input: " + str(rotated_inp))
print("Rotated input horizontal: " + str(rotated_inp_horizontal))
print("Rotated input vertical: " + str(rotated_inp_vertical))



# Plot axes
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis

# Plot input and its rotations
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="#1A4A1A") # input
ax1.quiver(0,0,0,inp_first_rot[0],inp_first_rot[1],inp_first_rot[2],color="#1EAA1E") # input first rotated
ax1.quiver(0,0,0,inp_second_rot[0],inp_second_rot[1],inp_second_rot[2],color="#BAEEBA") #"#14E714") # input second rotated
ax1.quiver(0,0,0,rotated_inp[0],rotated_inp[1],rotated_inp[2],color="yellow") # input rotated

# Plot reference and its rotations
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="#0000B2") # reference
ax1.quiver(0,0,0,ref_first_rot[0],ref_first_rot[1],ref_first_rot[2],color="#3A3AE7") # reference first rotated
ax1.quiver(0,0,0,ref_second_rot[0],ref_second_rot[1],ref_second_rot[2],color="#ADADF9") #"#6969F9") # reference second rotated
ax1.quiver(0,0,0,rotated_ref[0],rotated_ref[1],rotated_ref[2],color="violet") # reference rotated

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 3 - Inverse the angles that lead to the direction vector
Reference horizontal: -44.999999999999986
Reference vertical: -44.999999999999986
Input horizontal: -45.00000000000001
Input vertical: 1.2074182697257333e-06
Rotated reference: [0. 1. 0.]
Rotated reference horizontal: -0.0
Rotated reference vertical: 0.0
Rotated input: [0.70710678 1.         0.70710678]
Rotated input horizontal: 35.26438968275465
Rotated input vertical: 30.000000000000004


## Using the unifying method for multiple relative directions that does not consider reference roll

In [6]:
print("Using the unifying method for multiple relative directions - Does not consider reference roll")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Using the unifying method for multiple relative directions, single result - Does not consider reference roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')


# Create dummy data 
df = pd.DataFrame(columns=["inp_x","inp_y","inp_z","ref_x","ref_y","ref_z","final_x","final_y","final_z"])
df["inp_x"] = [0.5,0.5,0.5]
df["inp_y"] = [-0.2,-0.2,-0.2]
df["inp_z"] = [0.1,0.3,0.5]
df["ref_x"] = [0,-1,1]
df["ref_y"] = [-1,0,1]
df["ref_z"] = [1,-1,0]

# Calculate relative directions
final_x, final_y, final_z = create_relative_directions(df["inp_x"],df["inp_y"],df["inp_z"],df["ref_x"],df["ref_y"],df["ref_z"],method="anglediff_sphere_coords")
df["final_x"] = final_x
df["final_y"] = final_y
df["final_z"] = final_z


print("Resulting directions")
print(df)

# Plot single result
plot_idx = 2

ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,df["inp_x"][plot_idx],df["inp_z"][plot_idx],df["inp_y"][plot_idx],color="green") # input, change Unity coords to cartesian right hand
ax1.quiver(0,0,0,df["ref_x"][plot_idx],df["ref_z"][plot_idx],df["ref_y"][plot_idx],color="blue") # reference, change Unity coords to cartesian right hand
ax1.quiver(0,0,0,df["final_x"][plot_idx],df["final_z"][plot_idx],df["final_y"][plot_idx],color="red") # final, change Unity coords to cartesian right hand


plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()



   

  0%|          | 0/3 [00:00<?, ?it/s]

Using the unifying method for multiple relative directions - Does not consider reference roll
Creating relative directions, method: anglediff_sphere_coords


100%|██████████| 3/3 [00:00<00:00, 1120.67it/s]

Resulting directions
   inp_x  inp_y  inp_z  ref_x  ref_y  ref_z   final_x   final_y   final_z
0    0.5   -0.2    0.1      0     -1      1  0.898682  0.400082  0.179736
1    0.5   -0.2    0.3     -1      0     -1 -0.229416 -0.324443 -0.917663
2    0.5   -0.2    0.5      1      1      0 -0.345042 -0.872864  0.345042


## Using the unifying method for single relative direction that does not consider reference roll

In [7]:
print("Using the unifying method for single relative direction - Does not consider reference roll")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Using the unifying method for single relative direction - Does not consider reference roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# Input and reference 
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])

# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]


# Get output 
final_dir_unity = create_relative_direction(inp_dir_unity,ref_dir_unity,method="anglediff_sphere_coords",log_verbose=False)
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]


# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final rotated

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()


Using the unifying method for single relative direction - Does not consider reference roll


## Individual approaches that do not consider roll of reference
These are mathematically correct, however the roll of the reference direction is not directly taken into account/ assumed to be the up-vector. 

In [8]:
# %%timeit

# Version 2 - DOES NOT CONSIDER ROLL OF REFERENCE
# Basic idea
# Find rotations around z (cartesian, Unity y) and subsequently x (cartesian, Unity x) 
# that would rotate reference vector to forward direction (Unity 0,0,1). 
# Apply those rotations to reference vector, but also to input vector.
# Resulting rotated input vector is final direction relative to forward vector. 
print("Version 2 - Rotation of the unit sphere - Does not consider reference roll")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 2 - Rotation of the unit sphere - Does not consider reference roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# Input and reference 
#inp_dir_unity = np.array([1,-0.5,0.1])
#ref_dir_unity = np.array([-0.2,1,0.8])
#inp_dir_unity = np.array([-1,1.5,0.3])
#ref_dir_unity = np.array([-0.2,-1,0.8])
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])



# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]

# Calculate horizontal angle offset as angle between direction vector's projection on x,y plane (cartesian) and forward vector
# Alternatively, use calculate_thetas_from_unity_coords (but can only use horizontal value here, because projection would skew vertical value)
ref_horizontal = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0])

# Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if ref_dir_cartesian[2] < 0:
    ref_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if ref_dir_cartesian[0] < 0:
    ref_horizontal *= -1
    
# Calculate horizontal angle offset for input 
inp_horizontal = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0])

# Calculate vertical angle offset for input
inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if inp_dir_cartesian[0] < 0:
    inp_horizontal *= -1

print("Reference horizontal: " + str(ref_horizontal))
print("Reference vertical: " + str(ref_vertical))
print("Input horizontal: " + str(inp_horizontal))
print("Input vertical: " + str(inp_vertical))

# calculate rot matrices
# axis are for right-handed coordinate system
# angle inverted for vertical
horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle= ref_horizontal, angle_in_degree=True)
vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle= -1 *ref_vertical, angle_in_degree=True)


# Apply calculated rotations to reference direction, in order to rotate it to forward direction (Unity 0,0,1)
# First horizontal, then vertical! Rotation axis must align 
rotated_ref = ref_dir_cartesian.copy()
rotated_ref = np.dot(horizontal_rot_mat,rotated_ref)
rotated_ref = np.dot(vertical_rot_mat, rotated_ref)


# Calculate horizontal angle offset of rotated reference
rotated_ref_horizontal = angle_between([rotated_ref[0],rotated_ref[1],0],[0,1,0])

# Calculate vertical angle offset of rotated reference
rotated_ref_vertical = angle_between(rotated_ref,[rotated_ref[0],rotated_ref[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if rotated_ref[2] < 0:
    rotated_ref_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if ref_dir_cartesian[0] < 0:
    rotated_ref_horizontal *= -1

print("Rotated reference: " + str(rotated_ref))
print("Rotated reference horizontal: " + str(rotated_ref_horizontal))
print("Rotated reference vertical: " + str(rotated_ref_vertical))



# Apply calculated rotations to input direction
# Same order, as for reference. First horizontal, then vertical! Rotation axis must align 
rotated_inp = inp_dir_cartesian.copy()
rotated_inp = np.dot(horizontal_rot_mat,rotated_inp)
rotated_inp = np.dot(vertical_rot_mat, rotated_inp)


# Calculate horizontal angle offset of rotated input
rotated_inp_horizontal = angle_between([rotated_inp[0],rotated_inp[1],0],[0,1,0])

# Calculate vertical angle offset of rotated input
rotated_inp_vertical = angle_between(rotated_inp,[rotated_inp[0],rotated_inp[1],0])

# if z (cartesian) is negative, vertical angle is negative 
if rotated_inp[2] < 0:
    rotated_inp_vertical *= -1
    
# if x (cartesian) is negative, horizontal angle is negative 
if rotated_inp[0] < 0:
    rotated_inp_horizontal *= -1

print("Rotated input: " + str(rotated_inp))
print("Rotated input horizontal: " + str(rotated_inp_horizontal))
print("Rotated input vertical: " + str(rotated_inp_vertical))



# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,rotated_ref[0],rotated_ref[1],rotated_ref[2],color="lightblue") # reference rotated
ax1.quiver(0,0,0,rotated_inp[0],rotated_inp[1],rotated_inp[2],color="lightgreen") # input rotated

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 2 - Rotation of the unit sphere - Does not consider reference roll
Reference horizontal: 180.0
Reference vertical: -45.00000000000001
Input horizontal: -45.00000000000001
Input vertical: 1.2074182697257333e-06
Rotated reference: [1.22464680e-16 1.41421356e+00 2.22044605e-16]
Rotated reference horizontal: 0.0
Rotated reference vertical: 0.0
Rotated input: [ 1.         -0.70710678 -0.70710678]
Rotated input horizontal: 125.26438968275465
Rotated input vertical: -30.000000000000004


In [9]:
# %%timeit

# Version 1b - DOES NOT CONSIDER ROLL OF REFERENCE
# Same idea as 1a, but use sphere coordinates
print("Version 1b - Angle differences with sphere coordinates - Does not consider reference roll")


fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 1b - Angle differences with sphere coordinates - Does not consider reference roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# forward 
forward_dir_unity = np.array([0,0,1])

# inputs
#inp_dir_unity = np.array([1,1,1])
#ref_dir_unity = np.array([-1,-1,1])
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])

# generate random data as input and reference in (-1,1)
if False:
    inp_dir_unity = np.array([random.random() * 2 - 1 for _ in range(3)])
    ref_dir_unity = np.array([random.random() * 2 - 1 for _ in range(3)]) 

# compute cartesian right-handed representation 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]
forward_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([forward_dir_unity])[0]

#### 
#inp_dir_cartesian = np.array([0.4609944825876826, 0.1321670315678083, -0.4489406317631861])
#ref_dir_cartesian = np.array([-0.20632662366938836, -0.36921655868324166, 0.36551083215510016])
#inp_dir_cartesian = np.array([0, 1, 1])
#ref_dir_cartesian = np.array([0, 0, -1])



# compute spherical coordinates 
inp_dir_spherical = cart2sph(inp_dir_cartesian)
ref_dir_spherical = cart2sph(ref_dir_cartesian) 
forward_dir_spherical = cart2sph(forward_dir_cartesian)

# compute difference between ref and input, modulo and add to forward dir 
final_dir_spherical = (inp_dir_spherical[0] - ref_dir_spherical[0], inp_dir_spherical[1] - ref_dir_spherical[1], 1)
final_dir_spherical = (final_dir_spherical[0] % (np.pi * 2), final_dir_spherical[1] % (np.pi * 2), 1)
final_dir_spherical = (final_dir_spherical[0] + forward_dir_spherical[0], final_dir_spherical[1] + forward_dir_spherical[1],1)
final_dir_spherical = (final_dir_spherical[0] % (np.pi * 2), final_dir_spherical[1] % (np.pi * 2), 1)


# convert final direction back to cartesian coordinates
final_dir_cartesian = sph2cart(*final_dir_spherical)

# verify vectors and directions 
inp_vert_angle = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
inp_horiz_angle = angle_between([inp_dir_cartesian[0],inp_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

ref_vert_angle = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
ref_horiz_angle = angle_between([ref_dir_cartesian[0],ref_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

final_vert_angle = angle_between(final_dir_cartesian,[final_dir_cartesian[0],final_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
final_horiz_angle = angle_between([final_dir_cartesian[0],final_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)

# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vert_angle *= -1
if ref_dir_cartesian[2] < 0:
    ref_vert_angle *= -1
if final_dir_cartesian[2] < 0:
    final_vert_angle *= -1

# if x (cartesian) is negative, horizontal angle is negative 
if inp_dir_cartesian[0] < 0:
    inp_horiz_angle *= -1
if ref_dir_cartesian[0] < 0:
    ref_horiz_angle *= -1
if final_dir_cartesian[0] < 0:
    final_horiz_angle *= -1

# print verification infos 
print("Horizontal angles:")
print("\tInput: " + str(inp_horiz_angle))
print("\tReference: " + str(ref_horiz_angle))
print("\tFinal: " + str(final_horiz_angle))
print("Vertical angles:")
print("\tInput: " + str(inp_vert_angle))
print("\tReference:" + str(ref_vert_angle))
print("\tFinal: " + str(final_vert_angle))
print("Vectors:")
print("\tInput: " + str(inp_dir_cartesian))
print("\tReference: " + str(ref_dir_cartesian))
print("\tFinal: " + str(final_dir_cartesian))


# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 1b - Angle differences with sphere coordinates - Does not consider reference roll
Horizontal angles:
	Input: -45.00000000000001
	Reference: 180.0
	Final: 135.00000000000003
Vertical angles:
	Input: 1.2074182697257333e-06
	Reference:-45.00000000000001
	Final: 44.999999999999986
Vectors:
	Input: [-1, 1, 0]
	Reference: [0, -1, -1]
	Final: (0.5, -0.5000000000000001, 0.7071067811865475)


In [10]:
# %%timeit

# Version 1a - DOES NOT CONSIDER ROLL OF REFERENCE
#
# Basic idea
# Difference of elevation angle between reference and input 
# Difference of horizontal angle between reference and input 
# Apply angle difference to forward direction 
# Is likely not the same as rotating reference direction to forward direction 
# and applying same rotations to input  
print("Version 1a - Angle differences - Does not consider reference roll")

fig = plt.figure(figsize=(15,10))
fig.canvas.manager.set_window_title("Version 1a - Angle differences - Does not consider reference roll")
ax1 = fig.add_subplot(1,1,1, projection='3d')

# Input and reference 
#inp_dir_unity = np.array([1,-0.5,0.1])
#ref_dir_unity = np.array([-0.2,1,0.8])
inp_dir_unity = np.array([-1,0,1])
ref_dir_unity = np.array([0,-1,-1])

# To cartesian from Unity 
inp_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([inp_dir_unity])[0]
ref_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([ref_dir_unity])[0]

# Calculate thetas, can only use horizontal, since vertical depends on different plane than projected one 
inp_horizontal, _ = calculate_thetas_from_unity_coords(inp_dir_unity,ret_degree=True)
ref_horizontal, _ = calculate_thetas_from_unity_coords(ref_dir_unity,ret_degree=True)

# Calculate vertical angle offset as angle between direction vector and projection on x,y plane (cartesian)
inp_vertical = angle_between(inp_dir_cartesian,[inp_dir_cartesian[0],inp_dir_cartesian[1],0])
ref_vertical = angle_between(ref_dir_cartesian,[ref_dir_cartesian[0],ref_dir_cartesian[1],0])


# if z (cartesian) is negative, vertical angle is negative 
if inp_dir_cartesian[2] < 0:
    inp_vertical *= -1
if ref_dir_cartesian[2] < 0:
    ref_vertical *= -1


# Calculate angle offsets 
offset_horizontal = inp_horizontal - ref_horizontal
offset_vertical = inp_vertical - ref_vertical


print("Input horizontal: " + str(inp_horizontal))
print("Reference horizontal: " + str(ref_horizontal))
print("Offset horizontal: " + str(offset_horizontal)) # rotate around Unity y, Right-Handed cartesian z
print("Input vertical: " + str(inp_vertical))
print("Reference vertical: " + str(ref_vertical))
print("Offset vertical: " + str(offset_vertical)) # rotate around Unity/ Right-Handed cartesian x 

# calculate rot matrices
# axis are for right-handed coordinate system
# angle inverted for horizontal
horizontal_rot_mat = calculate_rotation_matrix(rotate_around_axis="z", angle= -1 * offset_horizontal, angle_in_degree=True)
vertical_rot_mat = calculate_rotation_matrix(rotate_around_axis="x", angle=offset_vertical, angle_in_degree=True)


# apply calculated relative rotations to forward Unity vector (0,0,1) (in right-handed cartesian (0,1,0)) to get final direction 
# first vertical, then horizontal! Rotation axis must align 
final_dir_unity = np.array([0,0,1])
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]
final_dir_cartesian = np.dot(vertical_rot_mat, final_dir_cartesian)
final_dir_cartesian = np.dot(horizontal_rot_mat,final_dir_cartesian)

# Verify final direction has offset values 
final_vertical = angle_between(final_dir_cartesian,[final_dir_cartesian[0],final_dir_cartesian[1],0]) # angle between vec and projection on xy-plane (cartesian)
final_horizontal = angle_between([final_dir_cartesian[0],final_dir_cartesian[1],0],[0,1,0]) # angle between projection on xy-plane and forward vector (cartesian)
print("---")
print("Final horizontal (should match offset horizontal): " + str(final_horizontal)) 
print("Final vertical (should match offset vertical): " + str(final_vertical))



# Plot
ax1.quiver(0,0,0,2,0,0,color="black") # axis
ax1.quiver(0,0,0,0,2,0,color="black") # axis
ax1.quiver(0,0,0,0,0,2,color="black") # axis
ax1.quiver(0,0,0,inp_dir_cartesian[0],inp_dir_cartesian[1],inp_dir_cartesian[2],color="green") # input
ax1.quiver(0,0,0,ref_dir_cartesian[0],ref_dir_cartesian[1],ref_dir_cartesian[2],color="blue") # reference
ax1.quiver(0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2],color="red") # final

plt.xlim(-2.5, 2.5)
plt.ylim(-2.5, 2.5)
ax1.set_zlim(-2.5,2.5)
ax1.set_xlabel("Unity X / Cartesian X")
ax1.set_ylabel("Unity Z / Cartesian Y")
ax1.set_zlabel("Unity Y / Cartesian Z")

ax1.scatter(0,0,0,marker=".",color="black",s=500)

plt.show()

Version 1a - Angle differences - Does not consider reference roll
Input horizontal: -45.0
Reference horizontal: 180.0
Offset horizontal: -225.0
Input vertical: 1.2074182697257333e-06
Reference vertical: -45.00000000000001
Offset vertical: 45.00000120741828
---
Final horizontal (should match offset horizontal): 135.0
Final vertical (should match offset vertical): 45.00000120741827


In [11]:
print(1/0)

ZeroDivisionError: division by zero

## Experimental & Untested below

In [ ]:

fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(1,1,1, projection='3d')

# from x,y,z to x,y,z
input_pts = np.array([[0,0,0,1,-1,-1]])
              #[0,0,0,1,-1,-1]])



# Use rotate from vector to vector method 

from_vec = input_pts[0][3:]
print("From vec pre scaling: " + str(from_vec))

from_vec = from_vec / np.linalg.norm(from_vec)
to_vec = np.array([0,0,1])
rot_mat_align = rotate_align(from_vec,to_vec)
print(rot_mat_align)

pts_new = np.dot(rot_mat_align,input_pts[0][3:])





print("From vec scaled: " + str(from_vec))
print(to_vec)

ax1.quiver(input_pts[:,0],input_pts[:,1],input_pts[:,2],input_pts[:,3],input_pts[:,4],input_pts[:,5])
ax1.quiver(0,0,0,pts_new[0],pts_new[1],pts_new[2],color="green")
plt.xlim(-2, 2)
plt.ylim(-2, 2)
ax1.set_zlim(0,2)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")

ax1.scatter(0,0,0,marker=".",color="red",s=500)

plt.show()




In [ ]:
## Offsets 

fig = plt.figure(figsize=(15,10))

ax1 = fig.add_subplot(1,1,1, projection='3d')


first_vec = np.array([[0,0,0,1,1,1]])
second_vec = np.array([[0,0,0,0,1,1]])
offset_one = np.array([first_vec[0] - second_vec[0]])

theta_z = 180 / 360 * 2 * np.pi 
rot_mat_z = np.array([
    [np.cos(theta_z),-np.sin(theta_z),0],
    [np.sin(theta_z),np.cos(theta_z),0],
    [0,0,1]
])


pts_new = np.dot(rot_mat_z, first_vec[0][3:])
third_vec = np.array([[0,0,0,pts_new[0],pts_new[1],pts_new[2]]]) # np.array([[0,0,0,-1,-1,-1]])
pts_new = np.dot(rot_mat_z, second_vec[0][3:])
fourth_vec = np.array([[0,0,0,pts_new[0],pts_new[1],pts_new[2]]]) # np.array([[0,0,0,0,-1,-1]])

print(third_vec)
offset_two = np.array([third_vec[0] - fourth_vec[0]])

ax1.quiver(first_vec[:,0],first_vec[:,1],first_vec[:,2],first_vec[:,3],first_vec[:,4],first_vec[:,5], color = "green")
ax1.quiver(second_vec[:,0],second_vec[:,1],second_vec[:,2],second_vec[:,3],second_vec[:,4],second_vec[:,5], color = "green")
ax1.quiver(offset_one[:,0],offset_one[:,1],offset_one[:,2],offset_one[:,3],offset_one[:,4],offset_one[:,5], color = "red")

ax1.quiver(third_vec[:,0],third_vec[:,1],third_vec[:,2],third_vec[:,3],third_vec[:,4],third_vec[:,5], color = "blue")
ax1.quiver(fourth_vec[:,0],fourth_vec[:,1],fourth_vec[:,2],fourth_vec[:,3],fourth_vec[:,4],fourth_vec[:,5], color = "blue")
ax1.quiver(offset_two[:,0],offset_two[:,1],offset_two[:,2],offset_two[:,3],offset_two[:,4],offset_two[:,5], color = "yellow")


plt.xlim(-2, 2)
plt.ylim(-2, 2)
ax1.set_zlim(0,2)
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")

ax1.scatter(0,0,0,marker=".",color="red",s=500)

plt.show()






In [ ]:
## 
## More rotations 


# Rotation around axis, right-hand rule 
theta_x = 90 / 360 * np.pi * 2 
theta_y = 90 / 360 * np.pi * 2 
theta_z = 45 / 360 * np.pi * 2 

rot_mat_x = np.array([
    [1,0,0],
    [0,np.cos(theta_x),-np.sin(theta_x)],
    [0,np.sin(theta_x),np.cos(theta_x)]
])
rot_mat_y = np.array([
    [np.cos(theta_y),0,np.sin(theta_y)],
    [0,1,0],
    [-np.sin(theta_y),0,np.cos(theta_y)]
])
rot_mat_z = np.array([
    [np.cos(theta_z),-np.sin(theta_z),0],
    [np.sin(theta_z),np.cos(theta_z),0],
    [0,0,1]
])

#pts_new = np.dot(rot_mat_z, input_pts[0][3:])
#pts_new = np.dot(rot_mat_x, pts_new)
#pts_new = np.dot(rot_mat_y, pts_new)



'''
# Create Rotation Matrix from rotation vector, perform angle shifts in rotation vector simultaneously
# Right hand rule 
rotvec = (-1 * offset_vertical,0,offset_horizontal)
#rotvec = (45,45,0) ## TODO 
r = R.from_rotvec(rotvec)
rot_mat_combined = r.as_matrix()
final_dir_unity = np.array([0,0,1])
final_dir_cartesian = unity_pts_to_right_handed_cartesian_coords([final_dir_unity])[0]
final_dir_cartesian = np.dot(rot_mat_combined, final_dir_cartesian)
final_dir_cartesian = np.array([0,0,0,final_dir_cartesian[0],final_dir_cartesian[1],final_dir_cartesian[2]])
'''


# Create Rotation Matrix from rotation vector, perform angle shifts in rotation vector simultaneously
# Right hand rule 
theta_x = 90 / 360 * 2 * np.pi
theta_z = 90 / 360 * 2 * np.pi
theta_y = 45 / 360 * 2 * np.pi 

rotvec = (theta_x,0,theta_z)

r = R.from_rotvec(rotvec)
rot_mat_combined = r.as_matrix()

#pts_new = np.dot(rot_mat_combined,input_pts[0][3:])

### More earlier references 

References
* https://vr.tobii.com/sdk/develop/unity/documentation/api-reference/core/#tobiixrgeteyetrackingdatatobiixr_trackingspace

* https://docs.unity3d.com/ScriptReference/Quaternion-eulerAngles.html 
"In Unity these rotations are performed around the Z axis, the X axis, and the Y axis, in that order." 

* https://forum.unity.com/threads/which-euler-angles-convention-used-in-unity.41114/#post-6828104
"[...] the applicable convention is "z-x-y (extrinsic rotations)"" 

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.from_euler.html
lower-case letters for extrinsic rotations 

More references/ different approach:
* https://forum.unity.com/threads/convert-a-vector3-containing-eulerangles-into-a-normalized-direction-vector3.124171/#post-846164 
"using Mathf;
 float elevation = Deg2Rad(euler.x);
 float heading = Deg2Rad(euler.y);
 Vector3 direction = new Vector3(Cos(elevation) * Sin(heading), Sin(elevation), Cos(elevation) * Cos(heading));"
 Code does not included needed additional minus yet
* Immediately below: 
"Alternatively: Quaternion.Euler(theVectorAngles) * Vector3.forward;" 
 

In [ ]:
# Other stuff 

# sth else https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d/897677#897677

'''
def rotate_align(v1,v2):
    
    #Untested!
    #Credit: https://gist.github.com/kevinmoran/b45980723e53edeb8a5a43c49f134724
    
    
    axis = np.cross(v1,v2)
    
    cosA = np.dot(v1,v2)
    
    k = 1.0 / (1.0 + cosA)

    
    result = np.array([[(axis[0] * axis[0] * k) + cosA, \
                        (axis[1] * axis[0] * k) - axis[2], \
                        (axis[2] * axis[0] * k) + axis[1]], \
                        [(axis[0] * axis[1] * k) + axis[2], \
                        (axis[1] * axis[1] * k) + cosA,     \
                        (axis[2] * axis[1] * k) - axis[0]], \
                        [(axis[0] * axis[2] * k) - axis[1], \
                        (axis[1] * axis[2] * k) + axis[0],  \
                        (axis[2] * axis[2] * k) + cosA]])
    return result
'''
